In [1]:
import pandas as pd
import os
import pickle as pkl
from pprint import pprint
from plotly import express as px
from lightgbm import LGBMRegressor

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.model.utils import qwk_obj

In [4]:
df = pd.DataFrame()
names = None

for fold in range(config.lgbm_n_folds):
    model: LGBMRegressor = pkl.load(open(f"output/LGBM/{fold}.pkl", "rb"))["model"]
    importances = model.feature_importances_

    if not names:
        names = model.feature_name_
        df = pd.concat([df, pd.DataFrame({"name": names})], axis=1)

    temp = pd.DataFrame({f"LGBM_{fold}": importances})
    df = pd.concat([df, temp], axis=1)

In [5]:
df.head(10)

,name,LGBM_0,LGBM_1,LGBM_2,LGBM_3,LGBM_4,LGBM_5,LGBM_6,LGBM_7,LGBM_8,...,LGBM_15,LGBM_16,LGBM_17,LGBM_18,LGBM_19,LGBM_20,LGBM_21,LGBM_22,LGBM_23,LGBM_24
0,deberta_m0_c0,173,153,95,120,132,117,116,135,99,...,71,63,131,108,123,121,106,152,112,129
1,deberta_m0_c1,119,107,66,88,101,84,93,91,92,...,63,82,100,89,92,91,88,87,96,91
2,deberta_m0_c2,122,97,102,114,110,104,113,101,105,...,89,84,119,96,99,101,87,122,105,97
3,deberta_m0_c3,97,91,68,68,70,61,74,75,58,...,42,52,79,73,71,62,60,86,58,71
4,deberta_m0_c4,94,82,44,66,73,64,69,53,54,...,36,39,63,62,61,67,51,79,59,65
5,deberta_m0_c5,61,38,22,33,42,23,22,33,24,...,24,15,53,25,36,30,24,40,27,35
6,deberta_m1_c0,93,70,33,64,69,54,55,58,53,...,31,31,66,52,59,47,40,55,56,54
7,deberta_m1_c1,81,90,61,63,49,54,57,53,42,...,33,44,51,45,39,53,53,49,53,50
8,deberta_m1_c2,104,117,77,89,88,94,91,89,91,...,56,69,87,81,83,77,83,97,89,96
9,deberta_m1_c3,65,62,38,42,51,42,47,56,37,...,48,43,61,46,54,61,45,57,44,54


In [6]:
df["importance"] = df.loc[:, "LGBM_0":].median(axis=1)
df.sort_values(by="importance", ascending=False, inplace=True)
df = df[["name", "importance"]]

## Overall

Top 10 most important features

In [7]:
df.head(10)

,name,importance
20,deberta_m3_c2,134.0
0,deberta_m0_c0,117.0
12,deberta_m2_c0,115.0
18,deberta_m3_c0,106.0
2,deberta_m0_c2,101.0
1,deberta_m0_c1,91.0
8,deberta_m1_c2,88.0
14,deberta_m2_c2,81.0
19,deberta_m3_c1,77.0
21,deberta_m3_c3,72.0


Top 10 least important features

In [8]:
df.tail(10)

,name,importance
163,tfidf_54,0.0
164,tfidf_55,0.0
166,tfidf_57,0.0
168,tfidf_59,0.0
169,tfidf_60,0.0
170,tfidf_61,0.0
171,tfidf_62,0.0
172,tfidf_63,0.0
174,tfidf_65,0.0
148,tfidf_39,0.0


## Broad Groups

In [9]:
def get_broad_group(name):
    group = name.split("_")[0]

    if group == "tfidf" and name.split("_")[1] == "count":
        return "tfidf_count"

    return group

In [10]:
df["broad_group"] = df["name"].map(get_broad_group)

In [11]:
px.box(
    df,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

From an high level, it looks like
* `score` probabilities from DeBERTa model has he highest importance.
* `Paragraph` and `sentence` features also seem to be quite important as well.
* `word` and `tfidf_count` seems to be the least important.

In [12]:
def get_count_type(name):
    return "_".join(name.split("_")[1:3])

In [13]:
df["count_type"] = df["name"].map(get_count_type)

## Broad Operations

In [14]:
def get_operation_type(name):
    last_word = name.split("_")[-1]
    if last_word in [
        "sum",
        "max",
        "mean",
        "min",
        "q1",
        "q3",
    ]:
        return last_word

    if name.find("len_goe") != -1:
        return "greater_than"
    elif name.find("len_l") != -1:
        return "less_than"
    return "None"

In [15]:
df["operation_type"] = df["name"].map(get_operation_type)

In [16]:
px.box(
    df,
    x="importance",
    color="operation_type",
    range_x=(df.importance.min(), df.importance.max()),
)

Broadly, `min`, `kurtosis`, `q1`, `first` are worthless, regardless of broad group.

### Greater Than

In [17]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [18]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [19]:
df[(df.operation_type == "greater_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
69,paragraph_sentence_count_len_goe_16,24.0,paragraph,sentence_count,greater_than
77,paragraph_error_count_len_goe_16,12.0,paragraph,error_count,greater_than


**Verdict**

1. For paragraphs
   1. Count only sentence and error. Upper limit be 15.
2. For sentences
   1. Count only error and word. Upper limit be 20.

### Less Than

In [20]:
px.box(
    df[df.operation_type == "less_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [21]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [22]:
df[(df.operation_type == "less_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
68,paragraph_sentence_count_len_l_16,36.0,paragraph,sentence_count,less_than
76,paragraph_error_count_len_l_16,20.0,paragraph,error_count,less_than


**Verdict**

1. For paragraphs: Count only sentence and error. Upper limit be 20.
2. For sentences: Count only error and word. Upper limit be 20.

## Paragraph Features

In [23]:
paragraph_features = df.loc[df.broad_group == "paragraph"]
print("Shape of paragraph features:", paragraph_features.shape)

Shape of paragraph features: (36, 5)


In [24]:
px.box(
    paragraph_features,
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [25]:
paragraph_features[
    (paragraph_features.count_type == "char_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
47,paragraph_char_count_sum,43.0,paragraph,char_count,sum
46,paragraph_char_count_max,34.0,paragraph,char_count,max
45,paragraph_char_count_mean,19.0,paragraph,char_count,mean
57,paragraph_char_count_q3,19.0,paragraph,char_count,q3
56,paragraph_char_count_q1,7.0,paragraph,char_count,q1


In [26]:
paragraph_features[
    (paragraph_features.count_type == "word_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
50,paragraph_word_count_sum,63.0,paragraph,word_count,sum
49,paragraph_word_count_max,22.0,paragraph,word_count,max
48,paragraph_word_count_mean,20.0,paragraph,word_count,mean
59,paragraph_word_count_q3,19.0,paragraph,word_count,q3
58,paragraph_word_count_q1,7.0,paragraph,word_count,q1


## TFIDF and Count

In [27]:
tfidf_features = df.loc[df.broad_group == "tfidf"]
print("Shape of tfidf features:", tfidf_features.shape)

Shape of tfidf features: (100, 5)


In [28]:
px.box(
    tfidf_features,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

TFIDF count features are worthless.

In [29]:
tfidf_features[
    (tfidf_features.broad_group == "tfidf") & (tfidf_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
135,tfidf_26,35.0,tfidf,26,None
109,tfidf_0,23.0,tfidf,0,None
110,tfidf_1,15.0,tfidf,1,None
116,tfidf_7,3.0,tfidf,7,None
176,tfidf_67,3.0,tfidf,67,None
165,tfidf_56,3.0,tfidf,56,None
117,tfidf_8,2.0,tfidf,8,None
149,tfidf_40,2.0,tfidf,40,None
141,tfidf_32,1.0,tfidf,32,None
187,tfidf_78,1.0,tfidf,78,None


## DeBERTa Features

In [30]:
deberta_features = df.loc[df.broad_group.isin(["deberta"])]
print("Shape of deberta features:", deberta_features.shape)

Shape of deberta features: (42, 5)


In [31]:
def get_model_num(name):
    return int(name.split("_")[1][1])


deberta_features["model"] = deberta_features.name.map(get_model_num)

/tmp/ipykernel_29150/962180890.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
px.box(
    deberta_features,
    x="importance",
    color="model",
    range_x=(df.importance.min(), df.importance.max()),
)

## Word Features

In [33]:
word_features = df.loc[df.broad_group.isin(["word"])]
print("Shape of word features:", word_features.shape)

Shape of word features: (60, 5)


In [34]:
def get_specific(name):
    return name.split("_")[1]

word_features["specific"] = word_features["name"].map(get_specific)

/tmp/ipykernel_29150/957994367.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
px.box(
    word_features,
    x="importance",
    color="specific",
    # range_x=(df.importance.min(), df.importance.max()),
)

## Grammar Features

In [36]:
grammar_features = df.loc[df.broad_group.isin(["grammar"])]
print("Shape of grammar features:", grammar_features.shape)

Shape of grammar features: (28, 5)


In [37]:
grammar_features["specific"] = grammar_features["name"].map(get_specific)

/tmp/ipykernel_29150/2923871788.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
px.box(
    grammar_features,
    x="importance",
    color="specific",
    # range_x=(df.importance.min(), df.importance.max()),
)